```
Score = 18.5
Key points: 
    (1) 'tf_efficientnet_b0_ns' pretrain model from timm library
    (2) Data Augmentation for images in the image dataset
    (3) Use both image (128 neurons) and 12 useful_features = ['Subject Focus', 'Eyes', 'Face',
        'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
        
        nn.Linear(model.classifier.in_features, 128) 
        nn.Dropout(0.1)
        out = nn.Linear(128 + 12, 1)    
```

# Import libs

### At "Add data": add tez-lib and timmaster to input folder so don't need "internet" on.
```
If having "internet" on
!pip install tez
!pip install timm   
```

In [ ]:
import sys
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")

In [ ]:
import os
import tez            # a simple pytorch trainer
from tez.callbacks import EarlyStopping
import albumentations # Fast image augmentation library
import pandas as pd
import cv2            # OpenCV 
import numpy as np
import timm           # (Unofficial) PyTorch Image Models
import torch
import torch.nn as nn
from sklearn import metrics, model_selection
from tqdm import tqdm
import random
# %matplotlib inline
# from matplotlib import pyplot as plt

# Hyperparameters

In [ ]:
'''
(batch_size, image_size): (64, 256), (16, 384)
'''
class Args:
    batch_size = 64
    image_size = 256
    epochs = 10 
    
path_input = r'../input/petfinder-pawpularity-score/'

# Augmentation

## For training and validation

In [ ]:

train_aug = albumentations.Compose(
    [
        albumentations.Resize(Args.image_size,
                              Args.image_size,
                              p=1),
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2,
            sat_shift_limit=0.2,
            val_shift_limit=0.2,
            p=0.5
        ),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1),
            contrast_limit=(-0.1, 0.1),
            p=0.5
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

valid_aug = albumentations.Compose(
    [
        albumentations.Resize(Args.image_size,
                              Args.image_size,
                              p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

## For Testing

In [ ]:
test_aug = albumentations.Compose(
    [
        albumentations.Resize(Args.image_size, Args.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

## 2. Dataset

In [ ]:
class Dataset:
    '''To get data of each image'''
    def __init__(self, 
                 image_paths, 
                 dense_features, 
                 targets, 
                 augmentations=None):
        '''
        :param dense_features: chosen feature columns
            dense_features = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
                              'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
        '''
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        '''
        :param item: a row index of one image data
        return: dic
            Eg. dic = {'image': tensor data of the image,
                       'features': a dense features corresponding to this image, 
                       'targets': Pawpularity of this image}        
        '''        
        image = cv2.imread(self.image_paths[item]) # Eg. image.shape = (960, 720, 3): W, H and C
        
        # as opencv loads in BGR format by default, we want to show it in RGB.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Converts an image from one color space to another
                                                       # image.shape = (960, 720, 3): C here order RGB
            
        if self.augmentations is not None: # refer to train_aug and valid_aug
            augmented = self.augmentations(image=image)
            image = augmented['image']
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32) # (0, 1, 2) -> (2, 0, 1)
                                                                  #  H, W, C   -> C, H, W  
        features = self.dense_features[item, :]  # item is row index. Ex. item=0 is the first image       
        targets = self.targets[item]  # value of Pawpularity of an image
        
        return {
            'image': torch.tensor(image, dtype=torch.float),  # np array to torch tensor
            'features': torch.tensor(features, dtype=torch.float), 
            'targets': torch.tensor(targets,  dtype=torch.float)            
        }        

## 3. Model

In [ ]:
class Model(tez.Model):
    '''
    Note: in tez: Dataset === pytorch requires
                  model class: nn.Module (pytorch) -> tez.Model
    '''
    def __init__(self, model_name):
        '''
        :param model_name: name of model for timm.create_model()
            Eg1. model_name = 'tf_efficientnet_b0_ns'
            Eg2. model_name = 'swin_large_patch4_window12_384'
        '''
        super().__init__()
        
        # model: EfficientNet
        self.model = timm.create_model(model_name, 
                                       pretrained = True, 
                                       in_chans = 3)
        
        # Linear(in_features=1280, out_features=128, bias=True). print model to see the keywords
        # Inside this model: (classifier): Linear(in_features=1280, out_features=1000, bias=True)
        # change it to:      (classifier): Linear(in_features=1280, out_features=128, bias=True)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128) 
        
        # Dropout(p=0.1, inplace=False)
        self.dropout = nn.Dropout(0.1)
        
        # Linear(in_features=140, out_features=1, bias=True)
        self.out = nn.Linear(128 + 12, 1)  # 12 is from dense features -> 128+12=140
        
        self.step_scheduler_after = 'epoch'
        
        # Other ways
        # self.dropout = nn.Dropout(0.1)
        # self.dense1 = nn.Linear(140, 64)
        # self.dense2 = nn.Linear(64, 1)
        
        
    def forward(self, image, features, targets=None):
        x = self.model(image)  # x.shape = torch.Size([1, 128]). Eg. [[-0.0453,....0.1661]]
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.out(x)
        
        # Other ways
        # x = self.model(image)
        # x = self.dropout(x)
        # x = torch.cat([x, features], dim=1)
        # x = self.dense1(x)
        # x = torch.relu(x)
        # x = self.dense2(x)
        
        if targets is not None:
            loss = nn.MSELoss()(x, targets.view(-1, 1))
            metrics = self.monitor_metrics(x, targets)
            return x, loss, metrics
        return x, 0, {}  
        
        
    def monitor_metrics(self, outputs, targets):
        outputs = outputs.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        
        rmse = metrics.mean_squared_error(y_true=targets, 
                                          y_pred=outputs, 
                                          squared=False # True returns MSE, False returns RMSE
                                         )
        return {'rmse': rmse}
    
    
    def fetch_scheduler(self):
        '''
        For Learning rate
        Set the learning rate of each parameter group using a cosine annealing schedule
        '''
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer,
            T_0 = 10,
            T_mult = 1,
            eta_min = 1e-6,
            last_epoch = -1
        )
        return sch
    
    
    def fetch_optimizer(self):
        ''' Adam optimization algo'''
        opt = torch.optim.Adam(
            self.parameters(),
            lr = 1e-4
        )
        return opt

# 4. train and evaluation of dataset and Run model

In [ ]:
def run(KFold=10):
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    
    # Get df and create kfold column to split train and validation dataset
    df = pd.read_csv(os.path.join(path_input, 'train.csv')).reset_index(drop = True)    
    df["kfold"] = [random.randint(0, KFold-1) for i in range(len(df))]
    
    for fold in range(KFold):
        df_train = df[df.kfold != fold].reset_index(drop=True)
        df_valid = df[df.kfold == fold].reset_index(drop=True)                           
        print(df_train.shape, df_valid.shape)
        
        train_dataset = Dataset(
            image_paths=[os.path.join(path_input, f'train/{img}.jpg') for img in df_train["Id"].values],
            dense_features=df_train[dense_features].values,
            targets=df_train.Pawpularity.values,
            augmentations=train_aug,
        )

        valid_dataset = Dataset(
            image_paths=[os.path.join(path_input, f'train/{img}.jpg') for img in df_valid["Id"].values] ,
            dense_features=df_valid[dense_features].values,
            targets=df_valid.Pawpularity.values,
            augmentations=valid_aug,
        )

        model = Model(model_name= 'tf_efficientnet_b0_ns') 

        es = EarlyStopping(
            monitor = 'valid_rmse',
            model_path = f'model_cv{fold}.bin',
            patience = 3,
            mode = 'min',
            save_weights_only = True
        )

        model.fit(
            train_dataset,
            valid_dataset = valid_dataset,
            train_bs = Args.batch_size,
            valid_bs = 2*Args.batch_size,
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
            epochs = Args.epochs,
            callbacks = [es],
            fp16 = True
        )

# RUN MODEL
run(KFold=5)

# 5. Make prediction

In [ ]:
def prediction(KFold=10):
    dense_features = [
            'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
        ]
    
    super_final_predictions = []
    for fold in range(KFold):   
        print(f'fold = {fold}')
        model = Model(model_name= 'tf_efficientnet_b0_ns')         
        model.load(f"../input/test-model-cv01/model_cv{fold}.bin", device="cuda", weights_only=True) 
        # model.load(f"/kaggle/working/model_cv{fold}.bin", device="cuda", weights_only=True) 

        df_test = pd.read_csv(os.path.join(path_input, 'test.csv'))
        test_img_paths = [os.path.join(path_input, f'test/{img}.jpg') for img in df_test["Id"].values] 
        
        test_dataset = Dataset(
            image_paths=test_img_paths,
            dense_features=df_test[dense_features].values,
            targets=np.ones(len(test_img_paths)),
            augmentations=test_aug
        )
        
        test_predictions = model.predict(test_dataset, 
                                         batch_size=2*Args.batch_size, 
                                         n_jobs=-1)

        final_test_predictions = []
        for preds in tqdm(test_predictions):
            final_test_predictions.extend(preds.ravel().tolist())
        # End of for preds
        super_final_predictions.append(final_test_predictions)
    # End of for fold
    super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
    df_test["Pawpularity"] = super_final_predictions
    df_test = df_test[["Id", "Pawpularity"]]
    df_test.to_csv("submission.csv", index=False)

# RUN PREDICTION
prediction(KFold=2)

In [ ]:
df_test

In [ ]:
'''
4128bae22183829d2b5fea10effdb0c3	    0.571777
1	43a2262d7738e3d420d453815151079e	3.466797
2	4e429cead1848a298432a0acad014c9d	0.738281
3	80bc3ccafcc51b66303c2c263aa38486	4.218750
4	8f49844c382931444e68dffbe20228f4	3.060547
5	b03f7041962238a7c9d6537e22f9b017	3.275391
6	c978013571258ed6d4637f6e8cc9d6a3	1.929688
7	e0de453c1bffc20c22b072b34b54e50f	1.901367
'''